In [64]:
import time
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as bs
from selenium import webdriver as wb
from IPython.display import display, HTML
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

In [65]:
dong = '화정동'
url = "https://new.land.naver.com/offices?ms=35.1433487,126.8801591,16&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL"
gcs = url[url.find("3"):url.find(",16")]

In [66]:
s = Service(ChromeDriverManager().install())
driver = wb.Chrome(service=s)
driver.maximize_window()
driver.get(url)
driver.execute_script("window.open('https://www.movable-type.co.uk/scripts/latlong-utm-mgrs.html');")

In [67]:
driver.switch_to.window(driver.window_handles[1])
driver.find_element(By.CSS_SELECTOR, "input#deg-format-d").click()
driver.find_element(By.CSS_SELECTOR, "input#precision-mm").click()

In [68]:
gcs_input = driver.find_element(By.CSS_SELECTOR, "input#latlon")
gcs_input.clear()
gcs_input.send_keys(gcs)
gcs_input.send_keys(Keys.RETURN)
time.sleep(0.1)
utm_output = driver.find_element(By.CSS_SELECTOR, "input#utm-coord").get_attribute("value").split(" ")

In [69]:
driver.switch_to.window(driver.window_handles[0])
time.sleep(3)

columns = ['동', '위도', '경도', '상가종류', '월세', '평수', '층수']
df = pd.DataFrame(columns=columns)

pin = driver.find_elements(By.CSS_SELECTOR, "a.map_cluster--mix:not(.is-outside)")

In [70]:
driver.refresh()

In [71]:
# for i in range(len(pin)):
for i in range(2):
    while 1:
        map_element = driver.find_element(By.CSS_SELECTOR, "div#map.map_panel")
        my_x = map_element.size['width'] / 2 + driver.find_element(By.CSS_SELECTOR, "div#listContents1").size['width']/2
        # my_x = map_element.size['width'] / 2
        my_y = map_element.size['height'] / 2

        time.sleep(1)
        pin = driver.find_elements(By.CSS_SELECTOR, "a.map_cluster--mix:not(.is-outside)")
        target_x = float(pin[i].get_attribute("style").split(" ")[-1].replace("px;", ""))
        target_y = float(pin[i].get_attribute("style").split(" ")[-3].replace("px;", ""))
        if abs(target_x) > 10000 or abs(target_y) > 10000:
            driver.refresh()
            time.sleep(3)
        else :
            break

    if target_x < 380 or target_y < -30:
        continue

    gap_x = target_x - my_x
    gap_y = target_y - my_y

    distance_x = gap_x * 100 / 52
    distance_y = gap_y * 100 / 52

    utm_x = float(utm_output[2]) + distance_x
    utm_y = float(utm_output[3]) - distance_y
    
    print(i+1)
    print(f"타깃 좌표 : x : {target_x}, y : {target_y}")
    print("52 N",utm_x, utm_y)

    pin[i].click()

    try:
        body = driver.find_element(By.CSS_SELECTOR, 'div.item_list.item_list--article')
        while True:
            body.send_keys(Keys.END)

            time.sleep(1)
            loader = driver.find_element(By.CSS_SELECTOR, 'div.loader')
    except:
        pass
    
    data = driver.find_elements(By.CSS_SELECTOR, "div.item")

    for j in range(len(data)):
        text_list = data[j].text.strip().split('\n')

        idx = 1 if text_list[0] == "공인중개사협회매물" else 0  # 조건에 따라 인덱스 설정

        split_values = text_list[idx + 2].split(', ')  # 공통적으로 필요한 값

        df.loc[i] = [
            dong,
            utm_x,
            utm_y,
            text_list[idx],
            text_list[idx + 1],
            split_values[0],
            split_values[1]
        ]

    time.sleep(2)
    pin[i].click()



time.sleep(1)
driver.switch_to.window(driver.window_handles[1])

for k in range(df.shape[0]):
    # UTM 좌표 입력 필드 가져오기
    utm_input = driver.find_element(By.CSS_SELECTOR, "input#utm-coord")

    # 기존 값 삭제 후 새로운 UTM 좌표 입력
    utm_input.clear()
    utm_input.send_keys("52 N " + str(df.loc[k, '위도']) + ' ' + str(df.loc[k, '경도']))
    utm_input.send_keys(Keys.RETURN)
    time.sleep(0.1)

    # 변환된 위도/경도 가져오기
    latlon_value = driver.find_element(By.CSS_SELECTOR, "input#latlon").get_attribute("value")

    latlon_value.find(", ")
    lat, long = latlon_value.split(", ")
    df.loc[k, '위도'] = float(lat[:-3])
    df.loc[k, '경도'] = float(long[:-3])

df.to_csv(f"./data/03_부동산/{dong}_부동산.csv", encoding='utf-8-sig', index=False)
df[['위도', '경도']].to_csv(f"./data/04_부동산_test/{dong}_test.csv", encoding='utf-8-sig', index=False, header=False)
driver.switch_to.window(driver.window_handles[0])



1
타깃 좌표 : x : 826.875, y : 340.291
52 N 306243.355 3891051.281076923
2
타깃 좌표 : x : 734.61, y : 223.746
52 N 306065.9223076923 3891275.406076923


In [ ]:
driver.switch_to.window(driver.window_handles[0])